In [1]:
# Import libraries used for collaborative filtering

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Flatten, Dense, Input, Concatenate
from tensorflow.keras.models import Model


2024-06-18 02:13:17.228078: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-18 02:13:17.270776: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 02:13:17.270816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 02:13:17.272721: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-18 02:13:17.282635: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Load the data
df = pd.read_csv('combined-dataset/final_reviews_dataV2.csv')

# Encode id and user id
user_enc = LabelEncoder()
df['user_id_encoded'] = user_enc.fit_transform(df['user_id'].values)

place_enc = LabelEncoder()
df['place_id_encoded'] = place_enc.fit_transform(df['id'].values)

n_users = df['user_id_encoded'].nunique()
n_places = df['place_id_encoded'].nunique()

print(f'Number of users: {n_users} and number of places: {n_places}')
df.head(10)

Number of users: 26496 and number of places: 6548


,id,types,review_number,review,user_id,sentiment,user_id_encoded,place_id_encoded
0,ChIJYcGr7GSb0S0RckePBrCWikw,"hotel, lodging",review 1,"It has quite small room, and the hallway is qu...",user_5061,0.012500,14590,3660
1,ChIJZbWX6Aia0S0R0tM3h1RZ1h8,"indonesian_restaurant, restaurant, food",review 1,"Surprisingly, a really good warung that’s hidd...",user_17031,0.212500,2528,3762
2,ChIJYyHbhgia0S0RzdjNXLmcf54,"tourist_attraction, restaurant, food",review 1,"Only had a fleeting visit here, came by coach,...",user_13759,-0.016667,1350,3678
3,ChIJ6zf9LJCb0S0RFv3BdLl61ZY,"coffee_shop, cafe, food, store",review 1,"One word, underrated! How come place like this...",user_5570,0.344676,16428,829
4,ChIJxaITmQia0S0RyrbukE8vsJU,"tourist_attraction, place_of_worship",review 1,"This temple is located in Singaraja, located i...",user_20325,0.139524,3720,6310
5,ChIJ63FmGgaa0S0RWD5dfwhjGHQ,"indonesian_restaurant, restaurant, food",review 1,"We came here for dinner, food was good, i like...",user_3693,0.186111,9687,746
6,ChIJAQAA5Aia0S0RBL27x0I5sHk,"indonesian_restaurant, restaurant, food",review 1,"They have 3 menus. Chicken, beef, or mixed. Th...",user_38971,-0.132630,10440,1202
7,ChIJA7v-3Qia0S0RdP8U7AQZBHE,"indonesian_restaurant, restaurant, food",review 1,"Did not meet my expectation...dirty table, not...",user_19070,-0.200000,3262,1193
8,ChIJnSg1VAia0S0R3r6ej3XRQ50,"breakfast_restaurant, indonesian_restaurant, r...",review 1,Super yum rawon (beef stew with spices)!\nDefi...,user_80954,0.470833,25537,5283
9,ChIJL_GkVA-a0S0R6UJc7dfTH4A,"restaurant, coffee_shop, cafe, bakery, wholesa...",review 1,A Singaraga Gem...\nHighly recommended...\nA f...,user_7798,0.308378,24462,2312


In [3]:
# Adjust if neccessary
embedding_size = 32

# User input
user_input = Input(shape=(1,), name='user_input')
user_embedding = Embedding(n_users, embedding_size, name='user_embedding')(user_input)
user_vec = Flatten(name='user_flatten')(user_embedding)

# Place input
place_input = Input(shape=(1,), name='place_input')
place_embedding = Embedding(n_places, embedding_size, name='place_embedding')(place_input)
place_vec = Flatten(name='place_flatten')(place_embedding)

# Concatenate user and place vectors
concat = Concatenate()([user_vec, place_vec])

# Add dense layers
dense_1 = Dense(128, activation='relu')(concat)
dense_2 = Dense(64, activation='relu')(dense_1)
output = Dense(1, activation='linear')(dense_2)

# Build the model
model = Model(inputs=[user_input, place_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Display the model summary
model.summary()

2024-06-18 02:13:21.170263: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 place_input (InputLayer)    [(None, 1)]                  0         []                            
                                                                                                  
 user_embedding (Embedding)  (None, 1, 32)                847872    ['user_input[0][0]']          
                                                                                                  
 place_embedding (Embedding  (None, 1, 32)                209536    ['place_input[0][0]']         
 )                                                                                            

2024-06-18 02:13:21.227521: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-18 02:13:21.227582: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-18 02:13:21.234415: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-18 02:13:21.234554: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-18 02:13:21.234592: I external/local_xla/xla/stream_executor

In [ ]:
# Train the model
history = model.fit(
    [df['user_id_encoded'], df['place_id_encoded']],
    df['sentiment'],
    epochs=10,
    batch_size=2,
    verbose=2
)


Epoch 1/10


2024-06-18 02:13:23.090787: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-06-18 02:13:23.532143: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f84ff292c40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-18 02:13:23.532193: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-06-18 02:13:23.538698: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-18 02:13:23.559518: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1718651603.625773  161753 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


16373/16373 - 152s - loss: 0.0560 - 152s/epoch - 9ms/step
Epoch 2/10
16373/16373 - 141s - loss: 0.0409 - 141s/epoch - 9ms/step
Epoch 3/10
16373/16373 - 134s - loss: 0.0284 - 134s/epoch - 8ms/step
Epoch 4/10
16373/16373 - 151s - loss: 0.0184 - 151s/epoch - 9ms/step
Epoch 5/10


In [ ]:
loss = model.evaluate([df['user_id_encoded'], df['place_id_encoded']], df['sentiment'])
print(f"Model Loss: {loss}")


In [ ]:
# model.save('collab_model.keras')

In [ ]:
# Test cell

# Function to recommend places for a given user
def recommend_places(user_id, model, df, user_encoder, place_encoder, top_n=5):
    user_idx = user_encoder.transform([user_id])[0]
    all_place_ids = df['place_id_encoded'].unique()
    user_array = np.array([user_idx] * len(all_place_ids))
    
    predictions = model.predict([user_array, all_place_ids])
    
    place_indices = predictions.flatten().argsort()[-top_n:][::-1]
    recommended_places = place_encoder.inverse_transform(place_indices)
    
    return recommended_places

# Get recommendations for a specific user
user_id = "user_18425"
recommended_places = recommend_places(user_id, model, df, user_enc, place_enc, top_n=10)
print(f"Recommended places for user {user_id}: {recommended_places}")

places = pd.read_csv('combined-dataset/combined_datasetV2.csv')
places = places[places['id'].isin(recommended_places)]  # Filter places
places
